In [1]:
# dependencies
import pandas as pd
import numpy as np
import pymongo
from pymongo import MongoClient

In [2]:
# import csvs

# cleaned data

nuke_file = pd.read_csv("data/cleaned_nuke.csv")

countries_file = pd.read_csv("data/countries.csv")

happy_file = pd.read_csv("data/suicide.csv")

In [3]:
# import to data bases
nuke_df = pd.DataFrame(nuke_file)

countries_df = pd.DataFrame(countries_file)

happy_df = pd.DataFrame(happy_file)

nuke_df.head()

,Unnamed: 0,Country,Code,Year,Quantity of Nuclear Weapons,Country_Year,GDP (Current LCU)
0,0,China,CHN,1960,0,China 1960,1483.49
1,1,China,CHN,1961,0,China 1961,1089.96
2,2,China,CHN,1962,0,China 1962,1020.73
3,3,China,CHN,1963,0,China 1963,1098.53
4,4,China,CHN,1964,1,China 1964,1268.46


In [4]:
countries_df

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [5]:
happy_df

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,Uzbekistan2014,0.675,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,Uzbekistan2014,0.675,"63,067,077,179",2309,Generation Z


In [6]:
# drop country column in countries_df
countries_df = countries_df.drop(columns=["country"])

# countries_df.head()

In [7]:
# renaming columns
countries_df = countries_df.rename(columns={"latitude":"Latitude", "longitude":"Longitude", "name":"Country"})
countries_df.head()

,Latitude,Longitude,Country
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


In [8]:
merge_ltlng = pd.merge(nuke_df, countries_df, on='Country')

merge_ltlng.head()

,Unnamed: 0,Country,Code,Year,Quantity of Nuclear Weapons,Country_Year,GDP (Current LCU),Latitude,Longitude
0,0,China,CHN,1960,0,China 1960,1483.49,35.86166,104.195397
1,1,China,CHN,1961,0,China 1961,1089.96,35.86166,104.195397
2,2,China,CHN,1962,0,China 1962,1020.73,35.86166,104.195397
3,3,China,CHN,1963,0,China 1963,1098.53,35.86166,104.195397
4,4,China,CHN,1964,1,China 1964,1268.46,35.86166,104.195397


In [9]:
# drop unnamed index from merge_ltlng

lat_merged_df = merge_ltlng.drop(columns=["Unnamed: 0"])

lat_merged_df.head()

,Country,Code,Year,Quantity of Nuclear Weapons,Country_Year,GDP (Current LCU),Latitude,Longitude
0,China,CHN,1960,0,China 1960,1483.49,35.86166,104.195397
1,China,CHN,1961,0,China 1961,1089.96,35.86166,104.195397
2,China,CHN,1962,0,China 1962,1020.73,35.86166,104.195397
3,China,CHN,1963,0,China 1963,1098.53,35.86166,104.195397
4,China,CHN,1964,1,China 1964,1268.46,35.86166,104.195397


In [10]:
# renaming columns for ease with database
lat_merged_df = lat_merged_df.rename(columns={"Quantity of Nuclear Weapons":"Quantity_of_Nuclear_Weapons",
                                            "GDP (Current LCU)":"GDP"})
lat_merged_df.head()


,Country,Code,Year,Quantity_of_Nuclear_Weapons,Country_Year,GDP,Latitude,Longitude
0,China,CHN,1960,0,China 1960,1483.49,35.86166,104.195397
1,China,CHN,1961,0,China 1961,1089.96,35.86166,104.195397
2,China,CHN,1962,0,China 1962,1020.73,35.86166,104.195397
3,China,CHN,1963,0,China 1963,1098.53,35.86166,104.195397
4,China,CHN,1964,1,China 1964,1268.46,35.86166,104.195397


In [11]:
happy_df = happy_df.rename(columns={"country-year":"Country_Year"})
happy_df["Country_Year"].unique() 



array(['Albania1987', 'Albania1988', 'Albania1989', ..., 'Uzbekistan2012',
       'Uzbekistan2013', 'Uzbekistan2014'], dtype=object)

In [12]:
# exporting a csv to combine with happiness indicators
# lat_merged_df.to_csv("data/nuke_coordinates_csv", index=False)

# Database Connection

In [13]:
#Define connection link an instantiate client
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

#Define the 'nukeDB' database in Mongo
db = client.nukeDB

#Declare the by_location collection
by_location = db.by_location

In [14]:
#Instantiate for loop for populating business collection
for index, row in lat_merged_df.iterrows():
    #Populate post dictionary with information to be inserted into business collection
    post = {
        "Country_Year": str(row["Country_Year"]),
        "Country": str(row["Country"]),
        "Code": str(row["Code"]),
        "Year": int(row["Year"]),
        "Quantity_of_Nuclear_Weapons": int(row["Quantity_of_Nuclear_Weapons"]),
        "GDP": float(row["GDP"]),
        "Latitude":float(row["Latitude"]),
        "Longitude":float(row["Longitude"])
    }
    #Insert data into business collection, post{} by post{}
    by_location.insert_one(post)